### 1. 라이브러리 불러오기 및 세션 생성

In [1]:
import json
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import explode, map_keys, col, first, get_json_object, array, to_json, struct


# Spark 세션 생성
spark = SparkSession \
        .builder \
        .appName("processingJson") \
        .getOrCreate()

23/11/17 17:36:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 2. 함수

In [2]:
# 분할된 JSON 파일 경로 선택하는 함수
def nth_json_path(n):
    return f'/Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta/naverplace_meta_{n}.json'


# get string dataframe!
def get_string_df(df, string_columns, string_df):    
    get_string_cols = df.select(string_columns)
    string_row = get_string_cols.filter(~col('name').isNull())
    return string_df.union(string_row)

### 3. 데이터 불러오기

In [3]:
# 첫번째 JSON 파일 데이터 불러오기
n = 1
data = spark.read.json(nth_json_path(n))

23/11/17 17:36:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### 4. 변수

In [4]:
columns = data.columns

In [5]:
hospital_bases = [c for c in columns if "HospitalBase" in c]

In [6]:
target_columns = [
    'name', 
    'road', 
    'reviewSettings', 
    'conveniences', 
    'keywords', 
    'phone', 
    'virtualPhone', 
    'naverBookingUrl', 
    'talktalkUrl', 
    'paymentInfo', 
    'homepages',
    'visitorReviewsTotal',
    'description',
    'Images'
]

In [7]:
string_columns = [
    'name', 
    'road',
    'phone',
    'virtualPhone',
    'naverBookingUrl',
    'talktalkUrl',
    'description'
]

In [8]:
struct_columns = [
    'reviewSettings',
    'homepages'
]

In [9]:
array_columns = [
    'conveniences',
    'keywords',
    'Images'
]

In [10]:
string_columns_schema = StructType([
    StructField("name", StringType(), True),
    StructField("road", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("virtualPhone", StringType(), True),
    StructField("naverBookingUrl", StringType(), True),
    StructField("talktalkUrl", StringType(), True),
    StructField("description", StringType(), True)
])

In [11]:
# struct_schema
# array_schema

In [12]:
string_df = spark.createDataFrame([], string_columns_schema)

### 5. 데이터 전처리

In [13]:
for hospital_base in hospital_bases:    
    hospital_base_data = data.select(hospital_base)
    
    get_columns = [col(hospital_base + "." + t).alias(t) for t in target_columns]
    df = hospital_base_data.select(get_columns)
    
    string_df = get_string_df(df, string_columns, string_df)

In [16]:
# string_df = string_df.toPandas()
# string_df

In [22]:
hospital_base = hospital_bases[0]

In [27]:
hospital_base_data = data.select(hospital_base)
get_columns = [col(hospital_base + "." + t).alias(t) for t in target_columns]
df = hospital_base_data.select(get_columns)
struct_df = df.select(struct_columns)
struct_df.pandas()

AttributeError: 'DataFrame' object has no attribute 'pandas'